# Imports

In [ ]:
# imports
import numpy as np
from tueplots import bundles, figsizes
import wandb
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import pandas as pd


import sys

%load_ext autoreload
%autoreload 2

sys.path.insert(0, '.')

In [ ]:
from analysis import sweep2df, plot_typography, stats2string


In [ ]:
USETEX = True

In [ ]:
plt.rcParams.update(bundles.neurips2022(usetex=USETEX))
# plt.rcParams.update({
#     'text.latex.preamble': [r'\usepackage{amsfonts}', # mathbb
#                             r'\usepackage{amsmath}'] # boldsymbol
# })

In [ ]:
plot_typography(usetex=USETEX, small=12, medium=16, big=20)

In [ ]:
# Constants
ENTITY = "causal-representation-learning"
PROJECT = "lti-ica"

# W&B API
api = wandb.Api(timeout=200)
runs = api.runs(ENTITY + "/" + PROJECT)

# Data loading

## Max variability

In [ ]:
SWEEP_ID = "6u3mgtpz"
sweep = api.sweep(f"{ENTITY}/{PROJECT}/{SWEEP_ID}")
filename = f"max_var_{SWEEP_ID}"
df_max_var, train_log_likelihood_max_var, train_mcc_max_var, val_log_likelihood_max_var, val_mcc_max_var = sweep2df(sweep.runs, filename, save=True, load=True)

### Max variability 10 dimensions

In [ ]:
SWEEP_ID = "woiubqya"
sweep = api.sweep(f"{ENTITY}/{PROJECT}/{SWEEP_ID}")
filename = f"max_var_10_{SWEEP_ID}"
df_max_var_10, train_log_likelihood_max_var_10, train_mcc_max_var_10, val_log_likelihood_max_var_10, val_mcc_max_var_10 = sweep2df(sweep.runs, filename, save=True, load=False)

### Rerun for 8 dimensions

In [ ]:
SWEEP_ID = "f2n0z65l"
sweep = api.sweep(f"{ENTITY}/{PROJECT}/{SWEEP_ID}")
filename = f"max_var_8_{SWEEP_ID}"
df_max_var_8, train_log_likelihood_max_var_8, train_mcc_max_var_8, val_log_likelihood_max_var_8, val_mcc_max_var_8 = sweep2df(sweep.runs, filename, save=True, load=True)

### Concatenate

In [ ]:
df_max_var_concat = pd.concat([df_max_var, df_max_var_8, df_max_var_10])

In [ ]:
df_max_var_concat.groupby(["num_comp", "zero_means", "use_B", "use_C"]).mean()[["train_mcc", "max_train_mcc", "val_mcc", "max_val_mcc"]]

In [ ]:
df_max_var_concat_dict = {comp: df_max_var_concat[df_max_var_concat.num_comp == comp].groupby(["num_comp", "zero_means", "use_B", "use_C"]) for
 comp in sorted(df_max_var_concat.num_comp.unique())}

In [ ]:
max_var_stats = [stats2string(df) for comp, df in df_max_var_concat_dict.items()]

## Minimal segments

### Original sweep

In [ ]:
SWEEP_ID =  "shrjtedq" #"03w02539"
sweep = api.sweep(f"{ENTITY}/{PROJECT}/{SWEEP_ID}")
filename = f"min_segment_{SWEEP_ID}"
df_min_segment, train_log_likelihood_min_segment, train_mcc_min_segment, val_log_likelihood_min_segment, val_mcc_min_segment = sweep2df(sweep.runs, filename, save=True, load=True)

### Rerun for 8 and 10 dimensions

In [ ]:
SWEEP_ID = "dvn24tw0"
sweep = api.sweep(f"{ENTITY}/{PROJECT}/{SWEEP_ID}")
filename = f"min_segment_8_10_{SWEEP_ID}"
df_min_segment_8_10, train_log_likelihood_min_segment_8_10, train_mcc_min_segment_8_10, val_log_likelihood_min_segment_8_10, val_mcc_min_segment_8_10 = sweep2df(sweep.runs, filename, save=True, load=True)

### Missing 8-/10-dimensional runs

In [ ]:
SWEEP_ID = "msankgos"
sweep = api.sweep(f"{ENTITY}/{PROJECT}/{SWEEP_ID}")
filename = f"min_segment_10_missing_{SWEEP_ID}"
df_min_segment_8_10_missing, train_log_likelihood_min_segment_8_10_missing, train_mcc_min_segment_8_10_missing, val_log_likelihood_min_segment_8_10_missing, val_mcc_min_segment_8_10_missing = sweep2df(sweep.runs, filename, save=True, load=True)

### Missing 10-dimensional runs

In [ ]:
SWEEP_ID = "v3bgbuna"
sweep = api.sweep(f"{ENTITY}/{PROJECT}/{SWEEP_ID}")
filename = f"min_segment_10_missing_{SWEEP_ID}"
df_min_segment_10_missing, train_log_likelihood_min_segment_10_missing, train_mcc_min_segment_10_missing, val_log_likelihood_min_segment_10_missing, val_mcc_min_segment_10_missing = sweep2df(sweep.runs, filename, save=True, load=True)

### Concatenate

In [ ]:
df_min_segment_concat = pd.concat([df_min_segment, df_min_segment_8_10, df_min_segment_8_10_missing, df_min_segment_10_missing])

In [ ]:
df_min_segment_concat.groupby(["num_comp", "zero_means", "use_B", "use_C"]).mean()[
    ["train_mcc", "max_train_mcc", "val_mcc", "max_val_mcc"]]

In [ ]:
df_min_segment_concat_dict = {
    comp: df_min_segment_concat[df_min_segment_concat.num_comp == comp].groupby(["num_comp", "zero_means", "use_B", "use_C"])
    for
    comp in sorted(df_min_segment_concat.num_comp.unique())}

In [ ]:
min_segment_stats = [stats2string(df) for comp, df in df_min_segment_concat_dict.items()]

# Render text for table

In [ ]:
all_stats = ["".join(["-----", str(comp),"\n" , min_segment, max_var, "\n"]) for comp, min_segment, max_var in zip(sorted(df_min_segment_concat.num_comp.unique()), min_segment_stats, max_var_stats)]

In [ ]:
print("\n".join(all_stats))